<a href="https://colab.research.google.com/github/yellowfleece/JHU-AI-Project1/blob/main/JHU_GENAI_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI Project 1 (Week 9)

## Project Instructions

### Business Context

As a business leader at Orion Tech Solutions, you (“Alex Carter”) oversee multiple software development and IT infrastructure projects. Your responsibilities include coordinating with stakeholders, managing escalations, and ensuring timely deliveries. With hundreds of emails flooding your inbox daily, manually sorting through them is time-consuming and increases the risk of missing critical updates, client escalations, or project approvals.

### Project Objective

Objective

The goal of this project is to develop a Generative AI-powered system that:

✅ Summarizes emails into actionable insights using the Yesterbox approach (excluding today’s emails).

✅ Prioritizes emails based on urgency, sender, and context.

✅ Draft context-aware responses to reduce manual effort.

✅ Evaluate the drafted context-aware responses using LLM-as-a-Judge.


### Tasks & Workflow

Task 1: Generate a Detailed Summary of Yesterday’s Inbox

Task 1A: Executive Dashboard (Top-Level Summary of Yesterday’s Emails)

Sample Output:

•	📧 Total Emails from Yesterday: 100

o	🚨 Urgent & High-Priority Emails: 10 (Require Immediate Action Today)

o	📝 Deadline-Driven Emails: 8 (Must Be Addressed Today)

o	🔄 Routine Updates & Check-ins: 35 (Review & Acknowledge)

o	📚 Non-Urgent & Informational Emails: 45 (Can Be Deferred or
Delegated)

o	💬 Personal & Social Emails: 22 (Optional Review)

o	🚫 Spam/Unimportant Emails Filtered Out: 20

AI Conclusion:

“You have 18 critical emails from yesterday that require action today. Additionally, there are 35 updates to review at your convenience.”

# 🔧 Phase 0: Environment Setup and CSV Data Loading

## Step 1: Environment Configuration

In [ ]:
# Install required packages
!pip install openai pandas python-dotenv chardet

In [2]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [12]:
# Configure OpenAI Key

import openai
openai.api_key = "OPEN_AI_KEY"

## Step 2: Load and Process CSV Data

### Load CSV with Encoding Detection (2A)

In [4]:
# =================================================================
# STEP 2A: LOAD CSV DATA WITH ENCODING DETECTION
# =================================================================

import pandas as pd
import chardet
from datetime import datetime

# Path to the provided CSV file
CSV_PATH = "/content/drive/MyDrive/email_assistant/Alex_emails_march_04.csv"

print(f"Loading emails from: {CSV_PATH}")

# Method 1: Try multiple encodings automatically
encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1', 'windows-1252']

df = None
successful_encoding = None

for encoding in encodings_to_try:
    try:
        print(f"Trying encoding: {encoding}")
        df = pd.read_csv(CSV_PATH, encoding=encoding)
        successful_encoding = encoding
        print(f"✅ Successfully loaded with {encoding} encoding!")
        break
    except UnicodeDecodeError:
        print(f"❌ {encoding} failed, trying next...")
        continue
    except Exception as e:
        print(f"❌ Error with {encoding}: {e}")
        continue

if df is not None:
    print(f"\n=== SUCCESSFUL DATA LOAD ===")
    print(f"✅ Successfully loaded {len(df)} emails from CSV")
    print(f"📧 Total columns: {len(df.columns)}")
    print(f"🔤 Encoding used: {successful_encoding}")

    # Display column names
    print(f"\n📋 Available columns:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i}. {col}")

    # Show basic dataset info
    print(f"\n📊 Dataset shape: {df.shape}")
    print(f"📅 First few rows:")
    display(df.head())

    # Check for missing values
    print(f"\n🔍 Missing values per column:")
    missing_counts = df.isnull().sum()
    for col, count in missing_counts.items():
        if count > 0:
            print(f"  {col}: {count} missing values")

else:
    print("❌ Failed to load CSV with any encoding. Please check the file path and format.")

Loading emails from: /content/drive/MyDrive/email_assistant/Alex_emails_march_04.csv
Trying encoding: utf-8
❌ utf-8 failed, trying next...
Trying encoding: latin-1
✅ Successfully loaded with latin-1 encoding!

=== SUCCESSFUL DATA LOAD ===
✅ Successfully loaded 60 emails from CSV
📧 Total columns: 6
🔤 Encoding used: latin-1

📋 Available columns:
  1. email_id
  2. date_received
  3. sender
  4. subject
  5. body
  6. main_recipient

📊 Dataset shape: (60, 6)
📅 First few rows:


,email_id,date_received,sender,subject,body,main_recipient
0,1,3/3/2025,Julia Martin,Approval Request: Budget Approval Needed by EOD,"Hi Alex,\n\nI hope you're doing well. As we ap...",Alex
1,2,3/3/2025,Fiona White,Are Your APIs Secure? Reddit & Discord Sound t...,"Hi Alex,\n\nA heated Discord discussion in the...",Alex
2,3,3/3/2025,Samantha Lee,Approval Needed: Project Scope Adjustment for ...,"Hi Alex,\n\nWeve encountered an unexpected AP...",Alex
3,4,3/3/2025,James Patel,Subject: Daily Update  Project Titan (March 3),"Hey Alex,\n\nQuick update on Project Titan for...",Alex
4,5,3/3/2025,David Whitmore,[URGENT] Dashboard Syncing Issues  Production...,"Hey Alex,\n\nWeve got a big issue right nowl...",Alex



🔍 Missing values per column:


### Standardize Column Structure (2B)

In [5]:
# =================================================================
# STEP 2B: STANDARDIZE COLUMN STRUCTURE
# =================================================================

print("=== STANDARDIZING EMAIL DATA STRUCTURE ===")

# The CSV columns are already well-named, so we just need to map to our standard names
print(f"Original columns: {list(df.columns)}")

# Rename columns to match our standard naming convention
df = df.rename(columns={
    'sender': 'from',
    'date_received': 'date',
    # 'subject' and 'body' are already correctly named
    # Keep email_id and main_recipient as they are
})

print(f"Standardized columns: {list(df.columns)}")

# Convert date column to proper datetime format
print(f"\n=== PROCESSING DATE COLUMN ===")
print(f"Sample dates before conversion: {df['date'].head(3).tolist()}")

# Convert date strings to datetime objects
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y', errors='coerce')

# Sort by date (most recent first)
df = df.sort_values('date', ascending=False).reset_index(drop=True)

print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Sample dates after conversion: {df['date'].head(3).tolist()}")

# Add enhanced metadata columns
print(f"\n=== ADDING METADATA COLUMNS ===")

# Extract sender domain
df['sender_domain'] = df['from'].str.split('@').str[1]

# Calculate text lengths
df['subject_length'] = df['subject'].str.len()
df['body_length'] = df['body'].str.len()

# Add day of week
df['day_of_week'] = df['date'].dt.day_name()

# Add category placeholder (will be filled by AI classification)
df['category'] = 'Unclassified'

# Reorder columns for better organization
column_order = ['email_id', 'date', 'day_of_week', 'from', 'sender_domain',
                'main_recipient', 'subject', 'subject_length', 'body',
                'body_length', 'category']

df = df[column_order]

print(f"✅ Enhanced dataset structure:")
print(f"📧 Total emails: {len(df)}")
print(f"📋 Final columns: {', '.join(df.columns)}")

# Display the enhanced dataset
print(f"\n=== ENHANCED DATASET PREVIEW ===")
display(df[['email_id', 'date', 'from', 'subject', 'category']].head())

=== STANDARDIZING EMAIL DATA STRUCTURE ===
Original columns: ['email_id', 'date_received', 'sender', 'subject', 'body', 'main_recipient']
Standardized columns: ['email_id', 'date', 'from', 'subject', 'body', 'main_recipient']

=== PROCESSING DATE COLUMN ===
Sample dates before conversion: ['3/3/2025', '3/3/2025', '3/3/2025']
Date range: 2025-03-03 00:00:00 to 2025-04-03 00:00:00
Sample dates after conversion: [Timestamp('2025-04-03 00:00:00'), Timestamp('2025-04-03 00:00:00'), Timestamp('2025-04-03 00:00:00')]

=== ADDING METADATA COLUMNS ===
✅ Enhanced dataset structure:
📧 Total emails: 60
📋 Final columns: email_id, date, day_of_week, from, sender_domain, main_recipient, subject, subject_length, body, body_length, category

=== ENHANCED DATASET PREVIEW ===


,email_id,date,from,subject,category
0,53,2025-04-03,Max Jeremy,Automate Your Software Development in 2025  A...,Unclassified
1,25,2025-04-03,Jake Dawson,Will Low-Code Replace Traditional Dev?,Unclassified
2,20,2025-04-03,Kevin Tran,Follow-up: Approval for Vendor API Licensing Fee,Unclassified
3,21,2025-04-03,Daniel Wong,Approval Request: Hiring a Senior Backend Deve...,Unclassified
4,23,2025-04-03,Rachel Thompson,[URGENT] Patient Data Sync Issues  Compliance...,Unclassified


### Data Quality Check (2C)

In [6]:
# =================================================================
# STEP 2C: DATA QUALITY CHECK
# =================================================================

print("=== DATA QUALITY ANALYSIS ===")

# Check for any issues with the data
print(f"📧 Email Statistics:")
print(f"  Total emails: {len(df)}")
print(f"  Unique senders: {df['from'].nunique()}")
print(f"  Unique domains: {df['sender_domain'].nunique()}")
print(f"  Date range: {(df['date'].max() - df['date'].min()).days + 1} days")

print(f"\n📊 Text Length Statistics:")
print(f"  Average subject length: {df['subject_length'].mean():.1f} characters")
print(f"  Average body length: {df['body_length'].mean():.1f} characters")
print(f"  Longest subject: {df['subject_length'].max()} characters")
print(f"  Longest body: {df['body_length'].max()} characters")

print(f"\n📅 Email Distribution by Day:")
day_counts = df['day_of_week'].value_counts()
for day, count in day_counts.items():
    print(f"  {day}: {count} emails")

print(f"\n🔍 Sample Email Preview:")
for i in range(min(3, len(df))):
    email = df.iloc[i]
    print(f"\n--- EMAIL {i+1} ---")
    print(f"From: {email['from']}")
    print(f"Subject: {email['subject']}")
    print(f"Date: {email['date']}")
    print(f"Body preview: {email['body'][:100]}...")

print(f"\n✅ Data quality check completed!")
print(f"✅ All {len(df)} emails ready for classification!")

# Save processed data as backup
csv_backup_path = "/content/drive/MyDrive/email_assistant/processed_emails_backup.csv"
df.to_csv(csv_backup_path, index=False)
print(f"✅ Backup saved to: {csv_backup_path}")

=== DATA QUALITY ANALYSIS ===
📧 Email Statistics:
  Total emails: 60
  Unique senders: 41
  Unique domains: 0
  Date range: 32 days

📊 Text Length Statistics:
  Average subject length: 51.9 characters
  Average body length: 444.4 characters
  Longest subject: 71 characters
  Longest body: 1083 characters

📅 Email Distribution by Day:
  Monday: 51 emails
  Thursday: 8 emails

🔍 Sample Email Preview:

--- EMAIL 1 ---
From: Max Jeremy
Subject: Automate Your Software Development in 2025  AI to the Rescue!
Date: 2025-04-03 00:00:00
Body preview: Hey Alex,

What if you could eliminate 80% of manual coding and speed up development by 5X? Our AI-p...

--- EMAIL 2 ---
From: Jake Dawson
Subject: Will Low-Code Replace Traditional Dev?
Date: 2025-04-03 00:00:00
Body preview: Hi Alex,

The Low-Code/No-Code movement is gaining traction, with Gartner predicting 65% of enterpri...

--- EMAIL 3 ---
From: Kevin Tran
Subject: Follow-up: Approval for Vendor API Licensing Fee
Date: 2025-04-03 00:00:00
Bod

# 🤖 Phase 1: Email Classification System Development

## Step 3: Email Classification Framework

### Write System Prompt for Classification (3A)

In [7]:
# =================================================================
# PHASE 1 - STEP 3A: EMAIL CLASSIFICATION SYSTEM PROMPT
# =================================================================

# <-- YOUR SYSTEM PROMPT GOES HERE -->
classification_system_prompt = """
You are an expert email classification assistant for Alex Carter, a business leader at Orion Tech Solutions who manages software development and IT infrastructure projects.

Your role is to analyze emails and classify them into exactly one of these 6 categories:

1. **Urgent & High-Priority**: Emails requiring immediate action today
   - Client escalations or complaints
   - Critical system failures or security issues
   - Executive requests requiring immediate response
   - Project blockers affecting deadlines

2. **Deadline-Driven**: Time-sensitive emails with specific deadlines that must be addressed today
   - Project deliverables due today/tomorrow
   - Meeting confirmations for today
   - Approval requests with tight deadlines
   - Time-sensitive contract or proposal responses

3. **Routine Updates**: Regular project updates, status reports, check-ins that need acknowledgment
   - Weekly/daily status reports
   - Progress updates from team members
   - Regular stakeholder communications
   - Scheduled check-ins

4. **Non-Urgent & Informational**: Can be deferred or delegated
   - Company newsletters or announcements
   - Documentation updates
   - Training materials
   - FYI communications

5. **Personal & Social**: Personal communications, social invitations, non-work related
   - Personal emails from friends/family
   - Social event invitations
   - Personal appointments
   - Non-work communications

6. **Spam/Unimportant**: Promotional emails, irrelevant content, emails to filter out
   - Marketing emails
   - Promotional content
   - Irrelevant communications
   - Obvious spam

Analyze the sender, subject line, content urgency, and business context. Respond with only the category name.
"""

print("✅ Classification system prompt created")
print(f"📏 Prompt length: {len(classification_system_prompt)} characters")

✅ Classification system prompt created
📏 Prompt length: 1726 characters


### Create Classification Function (3B)

In [8]:
# =================================================================
# PHASE 1 - STEP 3B: EMAIL CLASSIFICATION FUNCTION WITH RETRY LOGIC
# =================================================================

import time
from typing import Optional

def classify_email_with_ai(email_data, max_retries: int = 3) -> str:
    """
    Classify an email using OpenAI API with retry logic and better error handling

    Args:
        email_data: Email data dictionary
        max_retries: Maximum number of retry attempts

    Returns:
        Classification category string
    """
    # <-- YOUR USER PROMPT GOES HERE -->\n",
    user_prompt = f"""
    Please classify this email into one of the 6 categories.

    Email Details:
    From: {email_data['from']}
    Subject: {email_data['subject']}
    Date: {email_data['date']}
    Body: {email_data['body'][:500]}{"..." if len(email_data['body']) > 500 else ""}

    Respond with only one of these exact category names:
    - Urgent & High-Priority
    - Deadline-Driven
    - Routine Updates
    - Non-Urgent & Informational
    - Personal & Social
    - Spam/Unimportant
    """

    for attempt in range(max_retries):
        try:
            # Updated syntax for OpenAI 1.0+
            from openai import OpenAI
            client = OpenAI(api_key=openai.api_key)

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": classification_system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=50,
                temperature=0.1
            )

            classification = response.choices[0].message.content.strip()

            # Validate that we got one of the expected categories
            valid_categories = [
                'Urgent & High-Priority', 'Deadline-Driven', 'Routine Updates',
                'Non-Urgent & Informational', 'Personal & Social', 'Spam/Unimportant'
            ]

            if classification in valid_categories:
                return classification
            else:
                print(f"⚠️ Unexpected category '{classification}', defaulting to Non-Urgent & Informational")
                return "Non-Urgent & Informational"

        except Exception as e:
            print(f"❌ Attempt {attempt + 1} failed: {str(e)[:100]}...")

            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff: 1s, 2s, 4s
                print(f"⏳ Waiting {wait_time} seconds before retry...")
                time.sleep(wait_time)
                continue
            else:
                print(f"🔄 All {max_retries} attempts failed, using fallback classification")
                return "Non-Urgent & Informational"  # Safe fallback

print("✅ Enhanced email classification function with retry logic created")
print("🔧 Function includes exponential backoff and category validation")

✅ Enhanced email classification function with retry logic created
🔧 Function includes exponential backoff and category validation


### Test Classification System (3C)

In [13]:
# =================================================================
# PHASE 1 - STEP 3C: TEST CLASSIFICATION SYSTEM
# =================================================================

print("=== TESTING EMAIL CLASSIFICATION SYSTEM ===")
print("Testing on first 3 emails to verify API connectivity and accuracy\n")

# Test classification on first 3 emails
for i in range(min(3, len(df))):
    email = df.iloc[i]
    print(f"--- TESTING EMAIL {i+1} ---")
    print(f"From: {email['from']}")
    print(f"Subject: {email['subject']}")
    print(f"Body preview: {email['body'][:100]}...")

    # Classify the email
    category = classify_email_with_ai(email)
    print(f"🤖 AI Classification: {category}")

    # Update the DataFrame with test result
    df.loc[i, 'category'] = category
    print(f"✅ Updated in DataFrame\n")

print("🧪 Classification system test completed!")
print("📊 Ready for batch processing of all 60 emails")

=== TESTING EMAIL CLASSIFICATION SYSTEM ===
Testing on first 3 emails to verify API connectivity and accuracy

--- TESTING EMAIL 1 ---
From: Max Jeremy
Subject: Automate Your Software Development in 2025  AI to the Rescue!
Body preview: Hey Alex,

What if you could eliminate 80% of manual coding and speed up development by 5X? Our AI-p...
🤖 AI Classification: Spam/Unimportant
✅ Updated in DataFrame

--- TESTING EMAIL 2 ---
From: Jake Dawson
Subject: Will Low-Code Replace Traditional Dev?
Body preview: Hi Alex,

The Low-Code/No-Code movement is gaining traction, with Gartner predicting 65% of enterpri...
🤖 AI Classification: Non-Urgent & Informational
✅ Updated in DataFrame

--- TESTING EMAIL 3 ---
From: Kevin Tran
Subject: Follow-up: Approval for Vendor API Licensing Fee
Body preview: Hi Alex,

Just checking in on the API licensing request for SecurePay integration. We submitted this...
🤖 AI Classification: Deadline-Driven
✅ Updated in DataFrame

🧪 Classification system test completed

## Step 4: Batch Email Classification

In [14]:
# =================================================================
# PHASE 1 - STEP 4: BATCH EMAIL CLASSIFICATION WITH PROGRESS TRACKING
# =================================================================

print("=== CLASSIFYING ALL 60 EMAILS WITH ENHANCED ERROR HANDLING ===")
print("This will take approximately 3-5 minutes with API calls and retry logic...\n")

# Reset categories for fresh classification (except the 3 we already tested)
print("📋 Starting enhanced batch classification process")

# Track processing statistics
successful_classifications = 0
failed_classifications = 0
total_retries = 0

# Classify each email
for i in range(len(df)):
    # Skip the first 3 since we already classified them
    if i < 3:
        print(f"Email {i+1}/60: Already classified as '{df.loc[i, 'category']}'")
        successful_classifications += 1
        continue

    email = df.iloc[i]
    print(f"Classifying email {i+1}/60: {email['subject'][:50]}...")

    # Get AI classification with retry logic
    try:
        category = classify_email_with_ai(email)
        df.loc[i, 'category'] = category
        successful_classifications += 1
        print(f"  → {category}")
    except Exception as e:
        failed_classifications += 1
        print(f"  → ❌ Failed after all retries: {str(e)[:50]}...")

print(f"\n✅ Batch classification completed!")
print(f"📊 Processing Statistics:")
print(f"  ✅ Successful: {successful_classifications}/{len(df)}")
print(f"  ❌ Failed: {failed_classifications}/{len(df)}")
print(f"  📈 Success Rate: {(successful_classifications/len(df)*100):.1f}%")

# Show final classification summary
print(f"\n=== FINAL CLASSIFICATION SUMMARY ===")
category_counts = df['category'].value_counts()
for category, count in category_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{category}: {count} emails ({percentage:.1f}%)")

print(f"\n🎯 Enhanced classification complete! Ready for Phase 2: Executive Dashboard")

=== CLASSIFYING ALL 60 EMAILS WITH ENHANCED ERROR HANDLING ===
This will take approximately 3-5 minutes with API calls and retry logic...

📋 Starting enhanced batch classification process
Email 1/60: Already classified as 'Spam/Unimportant'
Email 2/60: Already classified as 'Non-Urgent & Informational'
Email 3/60: Already classified as 'Deadline-Driven'
Classifying email 4/60: Approval Request: Hiring a Senior Backend Develope...
  → Deadline-Driven
Classifying email 5/60: [URGENT] Patient Data Sync Issues  Compliance Ris...
  → Urgent & High-Priority
Classifying email 6/60: Can Someone Explain the Repeated Downtimes?...
  → Urgent & High-Priority
Classifying email 7/60: RE: Scheduling Q1 Executive Business Review (EBR) ...
  → Routine Updates
Classifying email 8/60: Is Your Cybersecurity Strategy Up to Date?...
  → Non-Urgent & Informational
Classifying email 9/60: URGENT: Overdue Invoice Approval for Cloud Service...
  → Urgent & High-Priority
Classifying email 10/60: Approval Reque

In [15]:
# =================================================================
# PHASE 1 - STEP 4: BATCH EMAIL CLASSIFICATION
# =================================================================

print("=== CLASSIFYING ALL 60 EMAILS ===")
print("This will take approximately 3-5 minutes with API calls...\n")

# Reset categories for fresh classification (except the 3 we already tested)
print("📋 Starting batch classification process")

# Classify each email
for i in range(len(df)):
    # Skip the first 3 since we already classified them
    if i < 3:
        print(f"Email {i+1}/60: Already classified as '{df.loc[i, 'category']}'")
        continue

    email = df.iloc[i]
    print(f"Classifying email {i+1}/60: {email['subject'][:50]}...")

    # Get AI classification
    category = classify_email_with_ai(email)
    df.loc[i, 'category'] = category

    print(f"  → {category}")

print("\n✅ All 60 emails classified!")

# Show classification summary
print("\n=== FINAL CLASSIFICATION SUMMARY ===")
category_counts = df['category'].value_counts()
for category, count in category_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{category}: {count} emails ({percentage:.1f}%)")

print(f"\n🎯 Classification complete! Ready for Phase 2: Executive Dashboard")

=== CLASSIFYING ALL 60 EMAILS ===
This will take approximately 3-5 minutes with API calls...

📋 Starting batch classification process
Email 1/60: Already classified as 'Spam/Unimportant'
Email 2/60: Already classified as 'Non-Urgent & Informational'
Email 3/60: Already classified as 'Deadline-Driven'
Classifying email 4/60: Approval Request: Hiring a Senior Backend Develope...
  → Deadline-Driven
Classifying email 5/60: [URGENT] Patient Data Sync Issues  Compliance Ris...
  → Urgent & High-Priority
Classifying email 6/60: Can Someone Explain the Repeated Downtimes?...
  → Urgent & High-Priority
Classifying email 7/60: RE: Scheduling Q1 Executive Business Review (EBR) ...
  → Routine Updates
Classifying email 8/60: Is Your Cybersecurity Strategy Up to Date?...
  → Non-Urgent & Informational
Classifying email 9/60: URGENT: Overdue Invoice Approval for Cloud Service...
  → Urgent & High-Priority
Classifying email 10/60: Approval Request: Budget Approval Needed by EOD ...
  → Deadline-Dri

In [16]:
# =================================================================
# CLASSIFICATION CONSISTENCY FIX
# =================================================================

print("=== CLEANING UP CLASSIFICATION INCONSISTENCIES ===")

# Show original category distribution
print("Original category distribution:")
original_counts = df['category'].value_counts()
for category, count in original_counts.items():
    print(f"  {category}: {count}")

def clean_category(category):
    """Standardize category names to remove formatting inconsistencies"""
    if 'Urgent & High-Priority' in str(category):
        return 'Urgent & High-Priority'
    elif 'Deadline-Driven' in str(category):
        return 'Deadline-Driven'
    elif 'Routine Updates' in str(category):
        return 'Routine Updates'
    elif 'Non-Urgent & Informational' in str(category):
        return 'Non-Urgent & Informational'
    elif 'Personal & Social' in str(category):
        return 'Personal & Social'
    elif 'Spam/Unimportant' in str(category):
        return 'Spam/Unimportant'
    else:
        return str(category)

# Clean up categories
df['category'] = df['category'].apply(clean_category)

# Show cleaned category distribution
print("\nCleaned category distribution:")
cleaned_counts = df['category'].value_counts()
for category, count in cleaned_counts.items():
    print(f"  {category}: {count}")

print(f"\n✅ Categories standardized! Total emails: {len(df)}")

=== CLEANING UP CLASSIFICATION INCONSISTENCIES ===
Original category distribution:
  Routine Updates: 19
  Urgent & High-Priority: 16
  Deadline-Driven: 13
  Spam/Unimportant: 7
  Non-Urgent & Informational: 5

Cleaned category distribution:
  Routine Updates: 19
  Urgent & High-Priority: 16
  Deadline-Driven: 13
  Spam/Unimportant: 7
  Non-Urgent & Informational: 5

✅ Categories standardized! Total emails: 60


In [17]:
# =================================================================
# CLASSIFICATION VALIDATION AND QUALITY CHECK
# =================================================================

def validate_classifications(dataframe):
    """Validate email classifications and report quality metrics"""

    print("=== CLASSIFICATION VALIDATION REPORT ===")

    # Check for valid categories
    valid_categories = [
        'Urgent & High-Priority', 'Deadline-Driven', 'Routine Updates',
        'Non-Urgent & Informational', 'Personal & Social', 'Spam/Unimportant'
    ]

    invalid_categories = dataframe[~dataframe['category'].isin(valid_categories)]['category'].unique()

    if len(invalid_categories) > 0:
        print(f"⚠️ Found {len(invalid_categories)} invalid categories:")
        for cat in invalid_categories:
            count = len(dataframe[dataframe['category'] == cat])
            print(f"  - '{cat}': {count} emails")
    else:
        print("✅ All categories are valid")

    # Check distribution balance
    category_counts = dataframe['category'].value_counts()
    critical_emails = len(dataframe[dataframe['category'].isin(['Urgent & High-Priority', 'Deadline-Driven'])])

    print(f"\n📊 Distribution Analysis:")
    print(f"  Critical emails: {critical_emails}/{len(dataframe)} ({critical_emails/len(dataframe)*100:.1f}%)")
    print(f"  Most common category: {category_counts.index[0]} ({category_counts.iloc[0]} emails)")

    # Quality indicators
    if critical_emails > len(dataframe) * 0.6:
        print("⚠️ Warning: >60% of emails are critical - may indicate over-classification")
    elif critical_emails < len(dataframe) * 0.1:
        print("⚠️ Warning: <10% of emails are critical - may indicate under-classification")
    else:
        print("✅ Critical email ratio appears balanced")

    return len(invalid_categories) == 0

# Run validation
validation_passed = validate_classifications(df)
print(f"\n{'✅ VALIDATION PASSED' if validation_passed else '❌ VALIDATION FAILED'}")

=== CLASSIFICATION VALIDATION REPORT ===
✅ All categories are valid

📊 Distribution Analysis:
  Critical emails: 29/60 (48.3%)
  Most common category: Routine Updates (19 emails)
✅ Critical email ratio appears balanced

✅ VALIDATION PASSED


# 📊 Phase 2: Task 1A - Executive Dashboard Creation

In [18]:
# =================================================================
# 📊 PHASE 2 - TASK 1A: EXECUTIVE DASHBOARD
# =================================================================

# <-- YOUR SYSTEM PROMPT GOES HERE -->
dashboard_system_prompt = """
You are an executive email assistant for Alex Carter at Orion Tech Solutions.

Your role is to create a professional executive dashboard summary that matches this exact format:

📧 Total Emails from Yesterday: [NUMBER]

🚨 Urgent & High-Priority Emails: [NUMBER] (Require Immediate Action Today)
📝 Deadline-Driven Emails: [NUMBER] (Must Be Addressed Today)
🔄 Routine Updates & Check-ins: [NUMBER] (Review & Acknowledge)
📚 Non-Urgent & Informational Emails: [NUMBER] (Can Be Deferred or Delegated)
💬 Personal & Social Emails: [NUMBER] (Optional Review)
🚫 Spam/Unimportant Emails Filtered Out: [NUMBER]

AI Conclusion:
"[Create a 1-2 sentence summary about critical emails requiring action today and routine items to review.]"

Use the exact emojis and format shown. Be professional and actionable.
"""

# Get clean category counts (handle the duplicate issue)
category_counts = df['category'].value_counts().to_dict()

# Clean up any duplicate entries
urgent_total = sum(count for category, count in category_counts.items()
                  if 'Urgent & High-Priority' in category)
deadline_total = category_counts.get('Deadline-Driven', 0)
routine_total = category_counts.get('Routine Updates', 0)
info_total = category_counts.get('Non-Urgent & Informational', 0)
personal_total = category_counts.get('Personal & Social', 0)
spam_total = category_counts.get('Spam/Unimportant', 0)

print(f"=== CLEAN CATEGORY COUNTS ===")
print(f"🚨 Urgent & High-Priority: {urgent_total}")
print(f"📝 Deadline-Driven: {deadline_total}")
print(f"🔄 Routine Updates: {routine_total}")
print(f"📚 Non-Urgent & Informational: {info_total}")
print(f"💬 Personal & Social: {personal_total}")
print(f"🚫 Spam/Unimportant: {spam_total}")
print(f"📧 Total: {urgent_total + deadline_total + routine_total + info_total + personal_total + spam_total}")

# <-- YOUR USER PROMPT GOES HERE -->
dashboard_user_prompt = f"""
Create an executive dashboard for yesterday's emails with these counts:

Email Classifications:
- Urgent & High-Priority: {urgent_total}
- Deadline-Driven: {deadline_total}
- Routine Updates: {routine_total}
- Non-Urgent & Informational: {info_total}
- Personal & Social: {personal_total}
- Spam/Unimportant: {spam_total}

Total emails: {len(df)}

Create the dashboard summary in the exact format specified in your system prompt.
"""

# Generate dashboard
try:
    from openai import OpenAI
    client = OpenAI(api_key=openai.api_key)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": dashboard_system_prompt},
            {"role": "user", "content": dashboard_user_prompt}
        ],
        max_tokens=300,
        temperature=0.1
    )

    dashboard_summary = response.choices[0].message.content.strip()
    print("\n=== EXECUTIVE DASHBOARD ===")
    print(dashboard_summary)

except Exception as e:
    print(f"Error generating dashboard: {e}")

print("\n✅ Task 1A: Executive Dashboard completed!")

=== CLEAN CATEGORY COUNTS ===
🚨 Urgent & High-Priority: 16
📝 Deadline-Driven: 13
🔄 Routine Updates: 19
📚 Non-Urgent & Informational: 5
💬 Personal & Social: 0
🚫 Spam/Unimportant: 7
📧 Total: 60

=== EXECUTIVE DASHBOARD ===
📧 Total Emails from Yesterday: 60

🚨 Urgent & High-Priority Emails: 16 (Require Immediate Action Today)
📝 Deadline-Driven Emails: 13 (Must Be Addressed Today)
🔄 Routine Updates & Check-ins: 19 (Review & Acknowledge)
📚 Non-Urgent & Informational Emails: 5 (Can Be Deferred or Delegated)
💬 Personal & Social Emails: 0 (Optional Review)
🚫 Spam/Unimportant Emails Filtered Out: 7

AI Conclusion:
"Yesterday, there were 16 urgent & high-priority emails and 13 deadline-driven emails that require immediate action today. Additionally, 19 routine updates need to be reviewed and acknowledged."

✅ Task 1A: Executive Dashboard completed!


# 🔥 Phase 3: Task 1B - Urgent Email Analysis

In [19]:
# =================================================================
# 🔥 PHASE 3 - TASK 1B: URGENT & HIGH-PRIORITY EMAIL ANALYSIS
# =================================================================

# <-- YOUR SYSTEM PROMPT GOES HERE -->
urgent_analysis_system_prompt = """
You are a business operations analyst for Alex Carter at Orion Tech Solutions.

Your role is to analyze urgent & high-priority emails and provide:
1. Clear summary of each email's key points
2. Immediate action requirements
3. Business impact assessment
4. Recommended next steps
5. Timeline for response

Focus on:
- Client relationships and escalations
- Critical project blockers
- Executive requests
- System issues affecting operations

Provide professional, actionable analysis that Alex can act on immediately.
"""

# Filter urgent emails
urgent_emails = df[df['category'].str.contains('Urgent & High-Priority', na=False)].copy()

print("=== TASK 1B: URGENT & HIGH-PRIORITY EMAIL ANALYSIS ===")
print(f"Found {len(urgent_emails)} urgent emails requiring immediate attention\n")

if len(urgent_emails) > 0:
    for i, (idx, email) in enumerate(urgent_emails.iterrows(), 1):
        print(f"--- URGENT EMAIL {i} ---")
        print(f"From: {email['from']}")
        print(f"Subject: {email['subject']}")
        print(f"Date: {email['date']}")

        # <-- YOUR USER PROMPT GOES HERE -->
        analysis_user_prompt = f"""
        Analyze this urgent email and provide actionable insights:

        From: {email['from']}
        Subject: {email['subject']}
        Date: {email['date']}
        Body: {email['body']}

        Provide:
        1. Key Points Summary
        2. Required Actions
        3. Business Impact
        4. Recommended Next Steps
        5. Response Timeline

        Be specific and actionable for a business leader.
        """

        try:
            from openai import OpenAI
            client = OpenAI(api_key=openai.api_key)

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": urgent_analysis_system_prompt},
                    {"role": "user", "content": analysis_user_prompt}
                ],
                max_tokens=400,
                temperature=0.1
            )

            analysis = response.choices[0].message.content.strip()
            print(f"\n🔥 URGENT ANALYSIS:")
            print(analysis)
            print("\n" + "="*50 + "\n")

        except Exception as e:
            print(f"Error analyzing urgent email: {e}")

else:
    print("✅ No urgent emails found - all clear for immediate priorities!")

print("✅ Task 1B: Urgent Email Analysis completed!")

=== TASK 1B: URGENT & HIGH-PRIORITY EMAIL ANALYSIS ===
Found 16 urgent emails requiring immediate attention

--- URGENT EMAIL 1 ---
From: Rachel Thompson
Subject: [URGENT] Patient Data Sync Issues  Compliance Risk
Date: 2025-04-03 00:00:00

🔥 URGENT ANALYSIS:
1. Key Points Summary:
   - Patient vitals data from the last 24 hours is missing in the Orion Health Monitor system.
   - 5 clinic locations are affected.
   - Doctors are currently unable to access crucial data for making medical decisions.
   - There is a potential compliance risk as this issue violates real-time monitoring requirements.

2. Required Actions:
   - Immediately investigate and resolve the patient data sync issues in the Orion Health Monitor system.
   - Provide an estimated time of resolution (ETA) to Rachel Thompson.
   - Ensure that missing patient vitals data is restored accurately and promptly.

3. Business Impact:
   - Doctors relying on missing data may make uninformed decisions, potentially compromising p

# ⏰ Phase 4 - Task 1C: Deadline-Driven E-mail Analysis

In [20]:
# =================================================================
# ⏰ PHASE 4 - TASK 1C: DEADLINE-DRIVEN EMAIL ANALYSIS
# =================================================================

# <-- YOUR SYSTEM PROMPT GOES HERE -->
deadline_analysis_system_prompt = """
You are a project timeline coordinator for Alex Carter at Orion Tech Solutions.

Your role is to analyze deadline-driven emails and provide:
1. Deadline identification and urgency assessment
2. Project impact analysis
3. Resource requirements
4. Timeline recommendations
5. Risk mitigation strategies

Focus on:
- Project deliverable deadlines
- Meeting schedules and confirmations
- Approval workflows with time constraints
- Contract and proposal timelines

Provide time-focused, actionable analysis that ensures no deadlines are missed.
"""

# Filter deadline-driven emails
deadline_emails = df[df['category'] == 'Deadline-Driven'].copy()

print("=== TASK 1C: DEADLINE-DRIVEN EMAIL ANALYSIS ===")
print(f"Found {len(deadline_emails)} deadline-driven emails requiring timely response\n")

if len(deadline_emails) > 0:
    for i, (idx, email) in enumerate(deadline_emails.iterrows(), 1):
        print(f"--- DEADLINE EMAIL {i} ---")
        print(f"From: {email['from']}")
        print(f"Subject: {email['subject']}")
        print(f"Date: {email['date']}")

        # <-- YOUR USER PROMPT GOES HERE -->
        deadline_user_prompt = f"""
        Analyze this deadline-driven email and provide time-focused insights:

        From: {email['from']}
        Subject: {email['subject']}
        Date: {email['date']}
        Body: {email['body']}

        Provide:
        1. Deadline Identification
        2. Time Sensitivity Assessment
        3. Required Actions by When
        4. Project/Business Impact
        5. Recommended Timeline

        Focus on ensuring no deadlines are missed and proper time management.
        """

        try:
            from openai import OpenAI
            client = OpenAI(api_key=openai.api_key)

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": deadline_analysis_system_prompt},
                    {"role": "user", "content": deadline_user_prompt}
                ],
                max_tokens=400,
                temperature=0.1
            )

            analysis = response.choices[0].message.content.strip()
            print(f"\n⏰ DEADLINE ANALYSIS:")
            print(analysis)
            print("\n" + "="*50 + "\n")

        except Exception as e:
            print(f"Error analyzing deadline email: {e}")

else:
    print("✅ No deadline-driven emails found - no immediate time pressures!")

print("✅ Task 1C: Deadline-Driven Email Analysis completed!")

=== TASK 1C: DEADLINE-DRIVEN EMAIL ANALYSIS ===
Found 13 deadline-driven emails requiring timely response

--- DEADLINE EMAIL 1 ---
From: Kevin Tran
Subject: Follow-up: Approval for Vendor API Licensing Fee
Date: 2025-04-03 00:00:00

⏰ DEADLINE ANALYSIS:
1. Deadline Identification:
   - Deadline: Approval for Vendor API Licensing Fee
   - Original Submission Date: February 20, 2025
   - Deadline for Approval: March 6, 2025

2. Time Sensitivity Assessment:
   - Urgency: High
   - The deadline for approval is critical as it is before the contract expires. Failure to meet this deadline may result in compliance issues with the payment gateway and potential disruption to services.

3. Required Actions by When:
   - Confirm approval status immediately to meet the March 6 deadline.
   - If more details are needed, request them from the relevant stakeholders promptly.

4. Project/Business Impact:
   - Failure to secure approval by March 6 could lead to non-compliance with the payment gateway r

# 📝 Phase 5: Task 2 - AI Response Draft Generation

In [21]:
# =================================================================
# 📝 PHASE 5 - TASK 2: AI RESPONSE DRAFT GENERATION
# =================================================================

# <-- YOUR SYSTEM PROMPT GOES HERE -->
response_drafting_system_prompt = """
You are a professional email composer for Alex Carter, a business leader at Orion Tech Solutions.

Your role is to draft professional, context-aware email responses that:

1. **Structure**: Follow proper email format:
   - Salutation (Dear/Hi [Name])
   - Greeting to the sender
   - Body addressing all key points
   - Clear next steps/decisions/required actions
   - Professional closing (Thank you, Best regards, etc.)

2. **Tone**: Maintain appropriate professionalism based on:
   - Sender relationship (CEO, client, team member)
   - Urgency level (immediate action vs. acknowledgment)
   - Business context (crisis response vs. routine communication)

3. **Content**: Address all points from original email and provide:
   - Clear acknowledgment of the issue/request
   - Specific next steps with timelines
   - Resource allocation if needed
   - Follow-up commitments

4. **Completeness**: Ensure no important details are omitted and all questions are answered.

Write as Alex Carter would respond - authoritative, solution-focused, and professionally courteous.
"""

# Get critical emails (Urgent + Deadline-Driven)
critical_emails = df[df['category'].isin(['Urgent & High-Priority', 'Deadline-Driven'])].copy()

print("=== TASK 2: AI-GENERATED RESPONSE DRAFTS FOR CRITICAL EMAILS ===")
print(f"Found {len(critical_emails)} critical emails requiring response drafts\n")

if len(critical_emails) > 0:
    for i, (idx, email) in enumerate(critical_emails.iterrows(), 1):
        print(f"--- CRITICAL EMAIL {i} ---")
        print(f"From: {email['from']}")
        print(f"Subject: {email['subject']}")
        print(f"Category: {email['category']}")
        print(f"Date: {email['date']}")

        # <-- YOUR USER PROMPT GOES HERE -->
        response_user_prompt = f"""
        Draft a professional email response to this critical email from Alex Carter:

        ORIGINAL EMAIL:
        From: {email['from']}
        Subject: {email['subject']}
        Date: {email['date']}
        Body: {email['body']}

        DRAFT REQUIREMENTS:
        1. Proper email structure: Salutation → Greeting → Body → Next Steps → Closing
        2. Address all key points from the original email
        3. Provide specific next steps with timelines
        4. Maintain appropriate tone for the relationship and urgency
        5. Include clear commitments and follow-up actions

        Write the complete email response that Alex Carter should send.
        """

        try:
            from openai import OpenAI
            client = OpenAI(api_key=openai.api_key)

            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": response_drafting_system_prompt},
                    {"role": "user", "content": response_user_prompt}
                ],
                max_tokens=500,
                temperature=0.2
            )

            draft_response = response.choices[0].message.content.strip()
            print(f"\n📧 DRAFT RESPONSE:")
            print("-" * 60)
            print(draft_response)
            print("-" * 60)
            print("\n" + "="*70 + "\n")

        except Exception as e:
            print(f"Error drafting response: {e}")

else:
    print("✅ No critical emails found requiring response drafts!")

print("✅ Task 2: AI Response Drafts completed!")

=== TASK 2: AI-GENERATED RESPONSE DRAFTS FOR CRITICAL EMAILS ===
Found 29 critical emails requiring response drafts

--- CRITICAL EMAIL 1 ---
From: Kevin Tran
Subject: Follow-up: Approval for Vendor API Licensing Fee
Category: Deadline-Driven
Date: 2025-04-03 00:00:00

📧 DRAFT RESPONSE:
------------------------------------------------------------
Subject: Re: Follow-up: Approval for Vendor API Licensing Fee

Dear Kevin,

Thank you for bringing this to my attention. I appreciate your diligence in following up on the API licensing request for the SecurePay integration.

I have reviewed the details you provided regarding the API licensing fee of $5,000/year, necessary for payment gateway compliance. I apologize for the delay in approval, which is indeed crucial before the contract expires on March 6, 2025.

To expedite the process, I have authorized the approval for the Vendor API Licensing Fee. You can proceed with the necessary steps to ensure timely integration with SecurePay.

Please 

# ⚖️ Phase 6: Task 3 - LLM-as-a-Judge Evaluation

In [22]:
# =================================================================
# ⚖️ PHASE 6 - TASK 3: LLM-AS-A-JUDGE EVALUATION (CORRECTED)
# =================================================================

# UPDATED SYSTEM PROMPT with stricter formatting requirements
judge_system_prompt = """
You are an expert content evaluator specializing in business communication and AI-generated content quality assessment.

Your role is to evaluate AI-generated email management content using these criteria:

**EVALUATION CRITERIA:**
1. **Relevance (1-10)**: How well does the content address the input query or task?
2. **Clarity (1-10)**: How clear and understandable is the content?
3. **Actionability (1-10)**: Does the content provide clear next steps or actionable information?

**MANDATORY EVALUATION FORMAT - FOLLOW EXACTLY:**

Relevance Score: [NUMBER 1-10]
Clarity Score: [NUMBER 1-10]
Actionability Score: [NUMBER 1-10]

Strengths:
- [Strength 1]
- [Strength 2]
- [Strength 3 if applicable]

Improvements:
- [Improvement 1]
- [Improvement 2 if applicable]

Overall Justification:
[Provide a 2-3 sentence summary evaluation with key observations]

YOU MUST follow this exact format. Do not use bold formatting, asterisks, or alternative formatting. Use only plain text with the exact labels shown above.
"""

# Content to evaluate (select key outputs from previous tasks)
evaluation_content = [
    {
        "task": "Executive Dashboard (Task 1A)",
        "description": "Top-level summary of yesterday's emails with category counts and AI conclusion",
        "sample_output": f"Dashboard showing {len(df)} total emails classified into 6 categories with executive summary and actionable insights for immediate priorities"
    },
    {
        "task": "Urgent Email Analysis (Task 1B)",
        "description": "Analysis of urgent & high-priority emails requiring immediate action",
        "sample_output": f"Analyzed {len(df[df['category'] == 'Urgent & High-Priority'])} urgent emails with detailed action recommendations, timelines, and business impact assessments"
    },
    {
        "task": "Email Response Drafts (Task 2)",
        "description": "AI-generated professional email responses for critical emails",
        "sample_output": f"Generated {len(df[df['category'].isin(['Urgent & High-Priority', 'Deadline-Driven'])])} response drafts with proper structure, professional tone, and clear next steps"
    }
]

print("=== TASK 3: LLM-AS-A-JUDGE EVALUATION (CORRECTED FORMAT) ===")
print("Evaluating AI-generated content for relevance, clarity, and actionability\n")

for i, content in enumerate(evaluation_content, 1):
    print(f"--- EVALUATION {i}: {content['task']} ---")

    # UPDATED USER PROMPT with explicit format requirements
    judge_user_prompt = f"""
    Evaluate this AI-generated business email management content:

    TASK: {content['task']}
    DESCRIPTION: {content['description']}
    OUTPUT RESULT: {content['sample_output']}

    CONTEXT: This is part of an email management system for Alex Carter, a business leader at Orion Tech Solutions managing software development and IT infrastructure projects.

    Please evaluate based on:
    1. Relevance: How well does this address the business need for Alex Carter?
    2. Clarity: How clear and understandable is this for a business leader?
    3. Actionability: How actionable are the insights provided for immediate use?

    YOU MUST respond in exactly this format:

    Relevance Score: [NUMBER 1-10]
    Clarity Score: [NUMBER 1-10]
    Actionability Score: [NUMBER 1-10]

    Strengths:
    - [List 2-3 key strengths]

    Improvements:
    - [List 1-2 areas for improvement]

    Overall Justification:
    [Provide 2-3 sentences summarizing your evaluation]

    Do not use bold text, asterisks, or any other formatting. Use only plain text.
    """

    try:
        from openai import OpenAI
        client = OpenAI(api_key=openai.api_key)

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": judge_system_prompt},
                {"role": "user", "content": judge_user_prompt}
            ],
            max_tokens=400,
            temperature=0.1  # Low temperature for consistent formatting
        )

        evaluation = response.choices[0].message.content.strip()
        print(f"\n⚖️ EVALUATION RESULTS:")
        print(evaluation)

        # Optional: Parse and validate the format
        lines = evaluation.split('\n')
        relevance_line = next((line for line in lines if 'Relevance Score:' in line), None)
        clarity_line = next((line for line in lines if 'Clarity Score:' in line), None)
        actionability_line = next((line for line in lines if 'Actionability Score:' in line), None)

        if relevance_line and clarity_line and actionability_line:
            print(f"\n✅ Format validation: PASSED")
        else:
            print(f"\n⚠️ Format validation: Some scores may be missing proper format")

        print("\n" + "="*60 + "\n")

    except Exception as e:
        print(f"Error in evaluation: {e}")

print("✅ Task 3: LLM-as-a-Judge Evaluation completed with corrected formatting!")

=== TASK 3: LLM-AS-A-JUDGE EVALUATION (CORRECTED FORMAT) ===
Evaluating AI-generated content for relevance, clarity, and actionability

--- EVALUATION 1: Executive Dashboard (Task 1A) ---

⚖️ EVALUATION RESULTS:
Relevance Score: 9
Clarity Score: 8
Actionability Score: 9

Strengths:
- The content provides a relevant top-level summary of yesterday's emails with category counts tailored for a business leader like Alex Carter.
- The inclusion of an executive summary and actionable insights enhances the understanding of the email data and its implications for immediate priorities.

Improvements:
- Clarifying the specific categories the emails are classified into would further enhance clarity.

Overall Justification:
The content is highly relevant to Alex Carter's need for a quick overview of email activity. It is mostly clear and actionable, offering valuable insights for immediate decision-making. Adding more clarity on the email categories would make it even more effective for Alex's use.

# 🚀 Phase 7: Task 4 - Analysis & Recommendations

## Project Observations

I found this project to be a good challenge of the skills that I have learned to date in this course as well as my own professional experience and personal learning of AI.

I felt as though it took me some time to get momentum on the project, but as I worked through some early challenges in setting up the environment, I felt like the tasks started to click and I was able to produce a working output per the project's instructions.

My key observations included:

**Strengths of the Use Case**
*   The use case has strong business value, as managing e-mail boxes and the requirement for urgent responses is something all executives and many employees face on a regular basis.
* The structured approach to the Project made sense to me, first building out an environment, then categorizing and prioritizng e-mails, then drafting responses, and finally doing an evaluation.
* Quality control was apparent through the "LLM-as-a-Judge" feature.  I have taken this similar approach in my personal/professional interaction with GPT chats; utilizing a different company's model altogether to evaluated generated outputs.
* Actionable outputs were obvious in the sense of the dashboard printout responses to the code built in the notebook

**Considerations of the Use Case**
* The provided CSV file is sufficient to run the use case, but obviously a different sampling from what might be encountered for different executive job roles in different industries.
* The pre-defined static categories may not have captured the nuance of different roles and/or industry as well
* The generative responses are likely to lack personal touch or tone of the specific individual; although there is likely opportunity to bolster this with more detailed approaches



## Suggestions for Improvement

Providing below some potential areas of improvement for the Project use case:

*  The existing system is focused primarily on analyzing text within e-mails which could limit its abililty to make nuanced decisions. An improvement would be an expanded analysis into e-mail metadata such as time patterns, attachment types, and thread length.  Mapping a sender relationship would also create more sophistication to the solution. These variables would better differentiate through routine e-mails and prioritization.

*  The system as developed operates using pre-determined and static rules that will not evolve in a sustained real world use case.  Implementing a feedback mechanism would allow the system to learn from miscategorized e-mails when an operator provides such feedback.  Over time the solution would improve accurancy and reduce manual reviews by the intended operator.


In [28]:
# Export to HTML per course instructions

# !pip install nbconvert

!jupyter nbconvert --to html "/content/sample_data/JHU_GENAI_Project1-v2.ipynb"

[NbConvertApp] Converting notebook /content/sample_data/JHU_GENAI_Project1-v2.ipynb to html
[NbConvertApp] Writing 577329 bytes to /content/sample_data/JHU_GENAI_Project1-v2.html
